In [2]:
import numpy as np
import pandas as pd

In [3]:
PATH_DATA = "V:/Programmazione/Amazon/"

In [8]:
df =  pd.read_csv(
    "%sdata.tsv" % PATH_DATA, 
    encoding="utf-8", 
    header=0,
    sep='\t',
    usecols=['review_id','customer_id', 'product_id', 'product_title','product_category','star_rating','helpful_votes','verified_purchase','review_headline','review_body','review_date'],
    index_col = 'review_id',
    dtype = {
        #'marketplace':str,
        'customer_id':str,
        'review_id':str,
        'product_id':str,
        #'product_parent':str,
        'product_title':str,
        'product_category':str,
        'star_rating': np.uint8,
        'helpful_votes': np.uint32,
        #'total_votes': np.uint32,
        #'vine':str,
        'verified_purchase':bool,
        'review_headline':str,
        'review_body':str
        },
    parse_dates = ['review_date'],
    true_values = ["Y"],
    false_values  = ["N"],
    skipinitialspace = True,
    engine='c'
    )

In [9]:
df.to_pickle("%sAmazonDataProject.pkl" % PATH_DATA)

In [10]:
df_no_text =  pd.read_csv(
    "%sdata.tsv" % PATH_DATA, 
    encoding="utf-8", 
    header=0,
    sep='\t',
    usecols=['review_id','customer_id', 'product_id', 'product_title','product_category','star_rating','helpful_votes','verified_purchase','review_date'],
    index_col = 'review_id',
    dtype = {
        #'marketplace':str,
        'customer_id':str,
        'review_id':str,
        'product_id':str,
        #'product_parent':str,
        'product_title':str,
        'product_category':str,
        'star_rating': np.uint8,
        'helpful_votes': np.uint32,
        #'total_votes': np.uint32,
        #'vine':str,
        'verified_purchase':bool,
        'review_headline':str,
        'review_body':str
        },
    parse_dates = ['review_date'],
    true_values = ["Y"],
    false_values  = ["N"],
    skipinitialspace = True,
    engine='c'
    )

In [11]:
df_no_text.to_pickle("%sAmazonDataProjectNoText.pkl" % PATH_DATA)

In [12]:
df_no_text.head()

,customer_id,product_id,product_title,product_category,star_rating,helpful_votes,verified_purchase,review_date
review_id,,,,,,,,
R2YVNBBMXD8KVJ,10349,B00MWK7BWG,My Favourite Faded Fantasy,Music,5,0,True,2014-12-29
R2K4BOL8MN1TTY,10629,B006CHML4I,Seiko 5 Men's Automatic Watch with Black Dial ...,Watches,4,0,True,2013-10-24
R3P40IEALROVCH,12136,B00IIFCJX0,Dexter Season 8,Digital_Video_Download,5,0,True,2014-05-09
R25XL1WWYRDLA9,12268,B000W7JWUA,The Settlers of Catan Board Game - discontinue...,Toys,5,0,True,2014-09-19
RVTVB9YDXSFYH,12677,B005JTAP4S,Peter: A Darkened Fairytale (Vol 1),Digital_Ebook_Purchase,5,12,False,2013-09-18
